In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 建立 Service 物件，指定 chromedriver.exe 的路徑
service = Service('./chromedriver.exe')

# 設定 Chrome 瀏覽器的選項
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized") # Chrome 瀏覽器在啟動時最大化視窗
options.add_argument("--incognito") # 無痕模式
options.add_argument("--disable-popup-blocking") # 停用 Chrome 的彈窗阻擋功能。

# 建立 Chrome 瀏覽器物件
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.104.com.tw/")


In [15]:
# 寫法 1 - 使用 CLASS_NAME
driver.find_element(By.CLASS_NAME, "form-control").send_keys("Python")

In [ ]:
# 寫法 2 - 使用 CSS_SELECTOR 找父層後再找子層
driver.find_element(By.CSS_SELECTOR, ".input-group.input-group--search").find_element(By.TAG_NAME, "input").send_keys("python")

In [ ]:
# 寫法 3 - 使用 CSS_SELECTOR 使用屬性精準匹配
driver.find_element(By.CSS_SELECTOR, "input[data-gtm-index='搜尋欄位-搜尋點擊']").send_keys("Python")

In [16]:
# 點擊搜尋按鈕
driver.find_element(By.CSS_SELECTOR, ".col.col-2").find_element(By.CSS_SELECTOR, "button[data-gtm-index='搜尋欄位-搜尋點擊']").click()

### 查找不同地區的資料

In [ ]:
import time
group_ele = driver.find_element(By.CLASS_NAME, "input-group-append")
group_ele.find_element(By.CSS_SELECTOR, "button[data-gtm-index='搜尋欄位-搜尋點擊']").click()
time.sleep(2)  # 等待頁面加載

# 取得所有區域選項的 value 以提供後續訪問各地區使用
checkbox_elements = driver.find_elements(By.CLASS_NAME, "category-picker-checkbox")
district_value = {}
for check_box_ele in checkbox_elements:
    district = check_box_ele.find_elements(By.TAG_NAME, "span")[1].text
    value = check_box_ele.find_element(By.TAG_NAME, "input").get_attribute("value")
    district_value[district] = value

print(f"區域選項數量：{len(checkbox_elements)}")

In [ ]:
# 確認抓取到的資料
district_value

In [ ]:
# 取得台北市的 value
area = district_value["台北市"]

# 造訪台北市的搜尋結果頁面
driver.get(f"https://www.104.com.tw/jobs/search/?area={area}&jobsource=index_s&keyword=python&mode=s&page=1")

In [ ]:
# 取得所有工作項目元素
job_elements = driver.find_elements(By.CSS_SELECTOR, ".vue-recycle-scroller__item-view.recycle-scroller--item")
print(f"工作項目數量：{len(job_elements)}")


In [ ]:
# 取得職缺資訊
for job_ele in job_elements:
    job_name = job_ele.find_element(By.CSS_SELECTOR, "a[data-gtm-joblist='職缺-職缺名稱']").text
    job_url = job_ele.find_element(By.CSS_SELECTOR, "a[data-gtm-joblist='職缺-職缺名稱']").get_attribute("href")
    company = job_ele.find_element(By.CSS_SELECTOR, "a[data-gtm-joblist='職缺-公司名稱']").text
    industry = job_ele.find_element(By.CSS_SELECTOR, "span[data-gtm-joblist^='職缺-產業']").text
    print(f"{job_name} | {job_url} | {company} | {industry}" )


In [ ]:
# 開啟新分頁
driver.execute_script("window.open('about:blank', '_blank');")

In [ ]:
# 取得所有分頁的 list
tabs = driver.window_handles
print(tabs)
# 切換至新分頁
driver.switch_to.window(tabs[-1])

In [ ]:
# 前往職缺詳細頁面
driver.get(job_url)

In [ ]:
# 關閉分頁
driver.close()

In [ ]:
# 關閉分頁後如未切回原分頁，會無法操作
# driver.get(job_url)

# 切回原分頁
driver.switch_to.window(tabs[0])

### 下滑至頁面底部，載入更多職缺

In [ ]:
job_elements = driver.find_elements(By.CSS_SELECTOR, ".vue-recycle-scroller__item-view.recycle-scroller--item")
print(f"工作項目數量：{len(job_elements)}")

# 執行下滑至頁面底部
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)  # 等待頁面加載

job_elements = driver.find_elements(By.CSS_SELECTOR, ".vue-recycle-scroller__item-view.recycle-scroller--item")
print(f"工作項目數量：{len(job_elements)}")